Módulo: Machine Learning<br>
Data: 12/2021<br>
Professor: Helder Vieira

# Aula 05 - Pipelines - Exercício

## Exercício 1

Quando estamos trabalhando com modelos, é muito comum que façamos o treinamento em um ambiente, mas precisamos executar o modelo em outro! O objetivo desse exercício é simular esse ambiente.

Suponha que uma área de uma concessionária de veículos deseja captar potenciais clientes. Para isso, os analistas da área farão contato com cada cliente potencial. A área considera que clientes potenciais são aqueles que ganham mais de $50K por ano.  

Por ser inviável o contato com todos os clientes, a empresa contratou dados com um fornecedor e deseja utilizá-los para priorizar o contato, focando em clientes com alta probabilidade de atenderem o requisito desejado.  

Refaça o exercício da aula 04 utilizando a Pipeline do sklearn. Salve o modelo treinado na pasta "Modelo Renda", onde se encontra o notebook para se fazer a inferência.  

### Carregando e visualizando os dados

In [1]:
import pandas as pd
import numpy as np

In [2]:
adult_train = pd.read_csv('adult_train.csv')

print(adult_train.shape)
display(adult_train.head(2))

(39074, 15)


,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,17,Private,144752,10th,6,Never-married,Handlers-cleaners,Own-child,Amer-Indian-Eskimo,Male,0,0,20,United-States,<=50K
1,21,Local-gov,402230,Some-college,10,Never-married,Adm-clerical,Unmarried,White,Male,0,0,36,United-States,<=50K


In [3]:
adult_train['income'] = adult_train['income'].map({'<=50K': 0, '>50K': 1})

In [4]:
categ_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']
num_features = ['age', 'capital-gain', 'capital-loss', 'hours-per-week']
special_features = ['educational-num']

In [5]:
X = adult_train[categ_features + num_features + special_features]
y = adult_train['income']

print(X.shape, y.shape)

(39074, 12) (39074,)


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, 
                                                    y, 
                                                    test_size=0.1, 
                                                    random_state=12)

print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

(35166, 12) (35166,)
(3908, 12) (3908,)


#### Construção de pipeline

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, FunctionTransformer
from lightgbm import LGBMClassifier
pd.options.mode.chained_assignment = None

In [9]:
# Função para processar os dados da variável educação
def transform_education(df):
    
    df = df.copy()
    
    df['educational-num'] = df['educational-num'].apply(lambda x: 1 if x >= 10 else 0)
    
    return df

In [10]:
# Configurando o categorizer
categorizer = ColumnTransformer(transformers = [
    # Processa as features categóricas usando OrdinalEncoder; -1 em caso de NaN
    ('transf_categ', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categ_features),
    # Processa as features numéricas com 'passthrough' em casos de NaN
    ('numeric', 'passthrough', num_features),
    # Processa a variável 'educational-num' chamando a função criada anteriormente
    ('education', FunctionTransformer(transform_education), ['educational-num'])   
])


In [11]:
# Construção da pipeline: executa categorizer e treino em sequência
pipe = Pipeline(steps = [
    ('categ', categorizer),
    ('modelo', LGBMClassifier(is_unbalance=True, learning_rate=0.1, max_depth=7, n_estimators=150, importance_type='gain')),
])

In [12]:
# Executando pipeline para treino do modelo
pipe.fit(X_train, y_train)

Pipeline(steps=[('categ',
                 ColumnTransformer(transformers=[('transf_categ',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['workclass',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'gender',
                                                   'native-country']),
                                                 ('numeric', 'passthrough',
                                                  ['age', 'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('education',
                  

In [13]:
# Executando pipeline para obter os preditos
y_train_pred = pipe.predict(X_train)

y_valid_pred = pipe.predict(X_valid)

print(y_train_pred.shape, y_valid_pred.shape)

(35166,) (3908,)


In [14]:
# Métricas do modelo treinado
from sklearn.metrics import accuracy_score, precision_score, recall_score

acc_train = accuracy_score(y_train, y_train_pred)
prec_train = precision_score(y_train, y_train_pred)
rec_train = recall_score(y_train, y_train_pred)

acc_valid = accuracy_score(y_valid, y_valid_pred)
prec_valid = precision_score(y_valid, y_valid_pred)
rec_valid = recall_score(y_valid, y_valid_pred)

print(f'Treino:\nAcc: {acc_train:.2f}, Precision: {prec_train:.2f}, Recall: {rec_train:.2f}')
print(f'Validação:\nAcc: {acc_valid:.2f}, Precision: {prec_valid:.2f}, Recall: {rec_valid:.2f}')

Treino:
Acc: 0.84, Precision: 0.62, Recall: 0.90
Validação:
Acc: 0.82, Precision: 0.57, Recall: 0.86


Temos um modelo para predizer se um indivíduo, a partir de suas características, recebe acima de 50 mil

### Salvando o modelo

#### Serialização do objeto

In [15]:
# Utilizamos a biblioteca pickle
import pickle

with open('modelo_adult.pkl', 'wb') as file: # 'wb' write bites
    pickle.dump(pipe, file)

### Uso do modelo

Como o modelo pode ser usado a partir do arquivo pickle 'modelo.pkl'

In [16]:
# 1: é preciso carregar funções que o programa utiliza
def transform_education(df):
    
    df = df.copy()
    
    df['educational-num'] = df['educational-num'].apply(lambda x: 1 if x >= 10 else 0)
    
    return df

In [17]:
# Carregando o arquivo
objeto_modelo = pickle.load(open('modelo_adult.pkl', 'rb')) # 'rb' read bites

In [18]:
objeto_modelo

Pipeline(steps=[('categ',
                 ColumnTransformer(transformers=[('transf_categ',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['workclass',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'gender',
                                                   'native-country']),
                                                 ('numeric', 'passthrough',
                                                  ['age', 'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('education',
                  

In [19]:
# usando o modelo: como exemplo, passando o df X_valid
objeto_modelo.predict(X_valid)

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

In [20]:
# Carregando um arquivo de teste
X_test = pd.read_csv('adult_test.csv')
X_test.shape

(9768, 15)

In [21]:
# Rodando o modelo
objeto_modelo.predict(X_valid)

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)